# Deliverable 3. Create a Travel Itinerary Map

----

1. Create a folder called `Vacation_Itinerary` to store all the files for this deliverable.

2. Download the `Vacation_Itinerary_starter_code.ipynb` file into your `Vacation_Itinerary` folder and rename it `Vacation_Itinerary.ipynb`.

3. Make sure the initial dependencies and the Geoapify API key are imported.

4. From your `Vacation_Search` folder from Deliverable 2, import the `WeatherPy_vacation.csv` file as a DataFrame named `vacation_df`.

5. Use GeoViews to create a map that shows all the cities in the `vacation_df` DataFrame. Configure the map as follows:

    * The point's size should be the maximum temperature for the city

    * The point's color should be the city's name

    * Use the `hover_cols` parameter to the the "Hotel Name", "Country", and "Current Description" columns to each point as additional information.

6. From the map, *choose four cities* that a customer might want to visit. They should be close together and in the same country. Use the `loc` method to create separate DataFrames for each city on the travel route.

    > **Hint:** You will start and end the route in the same city, so the `vacation_start` and `vacation_end` DataFrames will be in the same city.

7. Use the Pandas `concat` function to merge the DataFrame from each city in the itinerary to create a new DataFrame named `itinerary_df` to store the itinerary details.

8. Use the Pandas `copy` function to create a new DataFrame named `waypoints_df` to store the longitude and latitude for each city in `itinerary_df`.

    > **Hint:** You'll use this DataFrame to create a map using GeoViews, so recall that the first column should be the longitude, and the second the latitude.

9. Use GeoViews to create a map that shows the four cities in the itinerary.

10. Next, you'll use the Geoapify Routing API to find a route between the cities in the itinerary. Review the code that sets the initial parameters and fetches the coordinates from each city to define the `waypoints` parameter by using a `for` loop.

    > **Hint:** You can note that the `mode` parameter is set to `drive`, you can play around with other modes as it's shown in [the "Travel modes" table](https://apidocs.geoapify.com/docs/routing/#api) in the Geoapify Routing API documentation.

11. Use the Geoapify Routing API to retrieve the route's directions for your itinerary.

12. From the JSON response, store the route's legs coordinates in a variable called `legs`.

13. Loop through the route legs coordinates to fetch the latitude and longitude for each step. Store the latitude and longitude values into two Python lists named `longitude` and `latitude`.

14. Use the `longitude` and `latitude` Python lists to create a new DataFrame named `route_df`.

15. Use the GeoViews `Path` function to configure a line plot by using `route_df`. Set a custom color and width for the line that may contrast with the map.

16. Use the asterisk operator to display a composed plot that shows the itinerary's route over the map containing the cities. 

17. Save your map to the `Vacation_Itinerary` folder as `WeatherPy_travel_map.png`.

---

## Make sure the initial dependencies and the Geoapify API key are imported

In [22]:
# Dependencies and Setup
import geoviews as gv
import hvplot.pandas
import pandas as pd
import requests

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

# Import API key
from config import geoapify_key

# hide key
geoapify_key = ""

## From your `Vacation_Search` folder from Deliverable 2, import the `WeatherPy_vacation.csv` file as a DataFrame named `vacation_df`

In [2]:
# Read the WeatherPy_vacation.csv into a DataFrame
vacation_df = pd.read_csv("../Vacation_Search/WeatherPy_vacation.csv")

# Display sample data
vacation_df.head(50)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Morondava,-20.2833,44.2833,84.52,61,39,2.98,MG,2023-01-15 06:36:47
1,1,New Norfolk,-42.7826,147.0587,69.93,27,0,5.01,AU,2023-01-15 06:36:48
2,2,Barrow,71.2906,-156.7887,-0.38,71,40,20.71,US,2023-01-15 06:36:48
3,3,Alice Springs,-23.7000,133.8833,92.75,40,40,19.57,AU,2023-01-15 06:32:22
4,4,Karmaskaly,54.3709,56.1837,-4.09,90,99,9.48,RU,2023-01-15 06:36:49
5,5,Filadelfia,39.9523,-75.1638,31.10,69,75,16.11,US,2023-01-15 06:34:37
6,6,Abbeville,50.1000,1.8333,46.36,73,77,19.39,FR,2023-01-15 06:36:49
7,7,Busselton,-33.6500,115.3333,79.16,38,0,16.08,AU,2023-01-15 06:36:50
8,8,Comodoro Rivadavia,-45.8667,-67.5000,49.86,87,0,3.44,AR,2023-01-15 06:36:50
9,9,Thompson,55.7435,-97.8558,-0.24,85,100,8.05,CA,2023-01-15 06:36:51


## Use GeoViews to create a map that shows all the cities in the `vacation_df` DataFrame. Configure the map as follows:

* The point's size should be the maximum temperature for the city

* The point's color should be the city's name

* Use the `hover_cols` parameter to the the "Hotel Name", "Country", and "Current Description" columns to each point as additional information.

In [3]:
# Configure the map plot
# Configure the map plot
map_plot = vacation_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Max Temp",
    scale = 1,
    color = "City",
    hover_cols = ["Hotel Name", "County", "Weather Description"]
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp)

## From the map, *choose four cities* that a customer might want to visit. They should be close together and in the same country. Use the `loc` method to create separate DataFrames for each city on the travel route.

In [6]:
# Create DataFrames for each city by filtering the 'vacation_df' using the loc method
vacation_start = vacation_df.loc[vacation_df["City"] == "Ushuaia"]
vacation_end = vacation_df.loc[vacation_df["City"] == "Ushuaia"]
vacation_stop1 = vacation_df.loc[vacation_df["City"] == "Rio Gallegos"]
vacation_stop2 = vacation_df.loc[vacation_df["City"] == "Punta Arenas"]
vacation_stop3 = vacation_df.loc[vacation_df["City"] == "Coihaique"]

## Use the Pandas `concat` function to merge the DataFrame from each city in the itinerary to create a new DataFrame named `itinerary_df` to store the itinerary details

In [7]:
# Use the Pandas concat function to create a new DataFrame to store the itinerary details.
itinerary_df = pd.concat([vacation_start, vacation_end, vacation_stop1, vacation_stop2, vacation_stop3],ignore_index=True)

itinerary_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,38,Ushuaia,-54.8000,-68.3000,49.66,57,75,2.30,AR,2023-01-15 06:36:33
1,38,Ushuaia,-54.8000,-68.3000,49.66,57,75,2.30,AR,2023-01-15 06:36:33
2,183,Rio Gallegos,-51.6226,-69.2181,50.05,53,0,11.50,AR,2023-01-15 06:41:37
3,94,Punta Arenas,-53.1500,-70.9167,50.11,71,75,20.71,CL,2023-01-15 06:39:50
4,393,Coihaique,-45.5752,-72.0662,48.81,76,0,2.30,CL,2023-01-15 06:48:12


## Use the Pandas `copy` function to create a new DataFrame named `waypoints_df` to store the longitude and latitude for each city in `itinerary_df`

In [9]:
# Create a Pandas DataFrame to store the latitude and longitude for each city in the itineray
waypoints_df = itinerary_df[["Lat", "Lng"]]

# Display sample data
waypoints_df

,Lat,Lng
0,-54.8000,-68.3000
1,-54.8000,-68.3000
2,-51.6226,-69.2181
3,-53.1500,-70.9167
4,-45.5752,-72.0662


## Use GeoViews to create map that shows the four cities in the itinerary

In [10]:
# Configure the map plot by using the itineraty_df
waypoints_map = itinerary_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Max Temp",
    scale = 1,
    color = "City",
    hover_cols = ["Current Description"]
)

In [11]:
# Display the route_map
waypoints_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp)

## Next, you'll use the Geoapify Routing API to find a route between the cities in the itinerary. Review the code that sets the initial parameters and fetches the coordinates from each city to define the `waypoints` parameter by using a `for` loop

In [23]:
# Set parameters to trace the route
radius = 5000
params = {
    "mode":"drive",
    "apiKey": geoapify_key,
}

In [24]:
# Set an empty waypoints String variable
waypoints = ""

# Iterate through the route_df DataFrame to define the waypoints
for index, row in waypoints_df.iterrows():
    waypoints = waypoints + str(row["Lat"]) + "," + str(row["Lng"]) + "|"

# Delete the last character from the string
waypoints = waypoints[:-1]

# Add the waypoints to the params dictionary
params["waypoints"] = waypoints

# Display the params dictionary
params

{'mode': 'drive',
 'apiKey': '',
 'waypoints': '-54.8,-68.3|-54.8,-68.3|-51.6226,-69.2181|-53.15,-70.9167|-45.5752,-72.0662'}

## Use the Geoapify Routing API to retrieve the route's directions for your itinerary

In [14]:
# Set up the base URL for the Geoapify Places API.
base_url = "https://api.geoapify.com/v1/routing"

# Make request and retrieve the JSON data by using the params dictionaty
route_response = requests.get(base_url, params=params)


# Convert the API response to JSON format
route_response = route_response.json()
route_response

{'features': [{'type': 'Feature',
   'properties': {'mode': 'drive',
    'waypoints': [{'location': [-68.3, -54.8], 'original_index': 0},
     {'location': [-68.3, -54.8], 'original_index': 1},
     {'location': [-69.2181, -51.6226], 'original_index': 2},
     {'location': [-70.9167, -53.15], 'original_index': 3},
     {'location': [-72.0662, -45.5752], 'original_index': 4}],
    'units': 'metric',
    'distance': 2102895,
    'distance_units': 'meters',
    'time': 90012.9,
    'legs': [{'distance': 0,
      'time': 0,
      'steps': [{'from_index': 0,
        'to_index': 1,
        'distance': 0,
        'time': 0,
        'instruction': {'text': 'Drive north on Yaganes.'}},
       {'from_index': 1,
        'to_index': 1,
        'distance': 0,
        'time': 0,
        'instruction': {'text': 'Your destination is on the right.'}}]},
     {'distance': 586724,
      'time': 23605.917,
      'steps': [{'from_index': 0,
        'to_index': 5,
        'distance': 62,
        'time': 5.6

## From the JSON response, store the route's legs coordinates in a variable called `legs`

In [17]:
# Fetch the route's legs coordinates from the JSON reponse

legs = route_response['features'][0]['geometry']['coordinates']
print(legs)

[[[-68.301659, -54.800288], [-68.301659, -54.800288]], [[-68.301659, -54.800288], [-68.301815, -54.800065], [-68.301869, -54.79997], [-68.301899, -54.799911], [-68.30192, -54.799855], [-68.301944, -54.799752], [-68.301933, -54.799738], [-68.301926, -54.799718], [-68.301926, -54.799698], [-68.301903, -54.799649], [-68.301858, -54.79958], [-68.301802, -54.799511], [-68.301685, -54.799366], [-68.301642, -54.799343], [-68.3015, -54.79926], [-68.301389, -54.799203], [-68.301305, -54.799159], [-68.301212, -54.799112], [-68.301121, -54.799069], [-68.301015, -54.799024], [-68.300911, -54.798986], [-68.300796, -54.798945], [-68.300696, -54.798916], [-68.300569, -54.798885], [-68.300347, -54.798845], [-68.300142, -54.798811], [-68.299212, -54.798695], [-68.298707, -54.798647], [-68.298403, -54.798612], [-68.298235, -54.798595], [-68.298063, -54.798576], [-68.297992, -54.798567], [-68.296756, -54.798385], [-68.295551, -54.798231], [-68.29439, -54.798088], [-68.293945, -54.797986], [-68.293553, -5

## Loop through the route legs coordinates to fetch the latitude and longitude for each step. Store the latitude and longitude value into two Python lists names `longitude` and `latitude`

In [18]:
# Create and empty list to store the longitude of each step
Lng = []

# Create and empty list to store the latitude of step
Lat = []

# Loop through the legs coordinates to fetch the latitude and longitude for each step
for x in legs:
    for y in x:
        Lat.append(y[1])
        Lng.append(y[0])

print(Lat)
print(Lng)


[-54.800288, -54.800288, -54.800288, -54.800065, -54.79997, -54.799911, -54.799855, -54.799752, -54.799738, -54.799718, -54.799698, -54.799649, -54.79958, -54.799511, -54.799366, -54.799343, -54.79926, -54.799203, -54.799159, -54.799112, -54.799069, -54.799024, -54.798986, -54.798945, -54.798916, -54.798885, -54.798845, -54.798811, -54.798695, -54.798647, -54.798612, -54.798595, -54.798576, -54.798567, -54.798385, -54.798231, -54.798088, -54.797986, -54.797862, -54.797689, -54.797504, -54.797261, -54.796976, -54.796788, -54.796443, -54.796327, -54.796216, -54.796098, -54.796002, -54.795927, -54.795855, -54.795769, -54.795718, -54.795699, -54.795704, -54.795707, -54.795757, -54.795775, -54.795765, -54.795733, -54.795673, -54.795587, -54.79548, -54.795341, -54.795235, -54.795129, -54.794954, -54.794862, -54.794778, -54.794569, -54.794109, -54.793606, -54.793414, -54.793119, -54.793012, -54.792894, -54.792617, -54.792556, -54.792456, -54.79238, -54.792292, -54.792203, -54.792082, -54.7920

## Use the `longitude` and `latitude` Python lists to create a new DataFrame named `route_df`

In [19]:
# Create an empty DataFrame to store the steps' coordinates
route_df = pd.DataFrame()

# Add the steps' longitude and latitude from each step as columns to the DataFrame
route_df["Lat"] = Lat
route_df["Lng"] = Lng

# Display sample data
route_df

,Lat,Lng
0,-54.800288,-68.301659
1,-54.800288,-68.301659
2,-54.800288,-68.301659
3,-54.800065,-68.301815
4,-54.799970,-68.301869
...,...,...
15969,-45.575025,-72.069099
15970,-45.575051,-72.069014
15971,-45.575446,-72.067703
15972,-45.575869,-72.066296


##  Use the GeoViews `Path` function to configure a line plot by using `route_df`. Set a custom color and width for the line that may contrast with the map

In [20]:
# Configure the route path by using the GeoViews' Path function
route_path = gv.Path(route_df).opts(
    line_width=2, color='black', height=400, width=500
)
route_path

:Path   [Lat,Lng]

In [21]:
# Display a composed plot by using the route_map and route_path objects
waypoints_map * route_path

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp)
   .Path.I   :Path   [Lat,Lng]